In [424]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import numpy

In [447]:
# parse input fasta file
# extract sequences
def parseFasta(fileFasta):
    file = open(fileFasta) 
    lines = file.readlines()
    sequences = []
    for line in lines:
        line = line.rstrip()
        if not line.startswith(">"):
            sequences.append(line)
    return sequences

# strings must be of same length
def hammingDist(string1, string2):
    dist = 0
    length = len(string1)
    for i in range(length):
        if string1[i] != string2[i]:
            dist += 1
    return dist

In [448]:
# match query to all substrings of a sequence        
def minInSeq (i, query, substrings):
    strComp = substrings[i]
    dists = []
    while (strComp != "endSeq"):
        dists.append(hammingDist(query,strComp))
        i = i + 1
        strComp = substrings[i]
    minDist = min(dists)
    return minDist, i+1

# find hamming dist for set of substrings from a sequence 
def findHDsubstr (j, strScores, substrID, lenSubs, substrings):
    query = substrings[j]
    while (query != "endSeq"):
        i = 0 
        compSeqID = 0
        while (i < lenSubs):
            minDist, i = minInSeq(i,query,substrings)
            if compSeqID > 8721:
                print(compSeqID)
            strScores[substrID][compSeqID] = minDist
            compSeqID = compSeqID + 1
        j = j + 1
        query = substrings[j]
        substrID = substrID + 1
    return j + 1, strScores, substrID, compSeqID       

In [491]:
# finds the location of the motif in a sequence
# returns the match sequence
def locMotif (i, query, substrings):
    strComp = substrings[i]
    minDists = float('inf')
    foundMotif = ""
    while (strComp != "endSeq"):
        tempDist = hammingDist(query,strComp)
        if tempDist < minDists:
            foundMotif = substrings[i]
            minDists = tempDist
        i = i + 1
        strComp = substrings[i]
    return foundMotif, i+1

def findPWM (motif, matches):
    pwm = [[0] * 4 for i in range(len(motif))]
    for motInSeq in matches:
        index = 0
        for c in motInSeq:
            if c == 'A':
                pwm[index][0] = pwm[index][0] + 1
            if c == 'T':
                pwm[index][1] = pwm[index][1] + 1
            if c == 'C':
                pwm[index][2] = pwm[index][2] + 1
            if c == 'G':
                pwm[index][3] = pwm[index][3] + 1
            index = index + 1
    return pwm

def makeFreq (pwm, numSeqs):
    for x in range(0,len(pwm)):
        for y in range(0,4):
            pwm[x][y] = pwm[x][y] / numSeqs
    return pwm

In [483]:
# convert to .json
def writeJSON(jsonname, w,pwm):
    jfile = open(jsonname,'w') 
    jfile.write('[\n') 
    for i in range(0,w):
        writeWeights(i,pwm, jfile, w)
    jfile.write(']\n') 
    jfile.close() 
    
def writeWeights(i,pwm, jfile, w): #ACGT --> ATCG
    jfile.write('      {\n') 
    mystr = '             "' + 'A' + '": ' + str(pwm[i][0]) + ',\n'
    jfile.write(mystr)
    mystr = '             "' + 'T' + '": ' + str(pwm[i][1]) + ',\n'
    jfile.write(mystr)
    mystr = '             "' + 'C' + '": ' + str(pwm[i][2]) + ',\n'
    jfile.write(mystr)
    mystr = '             "' + 'G' + '": ' + str(pwm[i][3]) + '\n'
    jfile.write(mystr)
    if i < w:
        jfile.write('      },\n') 
    else:
        jfile.write('      }\n') 

In [499]:
def bruteMotifFinder (width, seq, jfile) :
    seqs = parseFasta(seq)
    # store all substr sequences of width w for each sequence
    substrings = []
    substrings2 =[]
    for seq in seqs:
        i = 0
        endSearch = len(seq) - width + 1
        while (i < endSearch):
            substrings.append(seq[i:i+width])
            substrings2.append(seq[i:i+width])
            i = i + 1
        substrings.append("endSeq")
        
    numSeqs = len(seqs)
    lenSubs = len(substrings)
    numSubstrs = len(substrings2)
    strScores = [[0] * numSeqs for i in range(numSubstrs)]
    j = 0
    substrID = 0
    while (j < lenSubs):
        j, strScores, substrID, compSeqID = findHDsubstr(j, strScores, substrID, lenSubs, substrings)
    
    # find total scores for each substr
    finScores = []
    for substr in range(0,len(strScores)):
        finScores.append(sum(strScores[substr]))
        
    # find substring with the minimal Hamming distance
    minHamDist = min(finScores)
    indexMinHD = finScores.index(minHamDist)
    motif = substrings2[indexMinHD]
    
    # now that we have the motif, get the PWM 
    # PWM in order of A,T,C,G

    # get the motif matches from each sequence
    k = 0
    matches = []
    while (k < lenSubs):
        foundMotif, k = locMotif(k, motif, substrings)
        matches.append(foundMotif)
        
    temppwm = findPWM(motif, matches)
    pwm = makeFreq(temppwm, numSeqs)
    #pwm
    
    #HOMEDIR = '/Users/vedranaivezic/Documents/Princeton/Princeton20-21/COS455/'
    #HOMEDIR = HOMEDIR + 'CS455_proj/'
    writeJSON(jfile, width,pwm)


In [510]:
HOMEDIR = '/Users/vedranaivezic/Documents/Princeton/Princeton20-21/COS455/'
HOMEDIR = HOMEDIR + 'CS455_proj/QCBFinalProject/tandem_repeat3/'
seq = HOMEDIR + 'tandrep3.fasta'
jsonFile = HOMEDIR + 'BruteSub.json'
width = 10

bruteMotifFinder(width, seq, jsonFile)

In [ ]:
# misc below

In [412]:
# set desired width
width = 10

In [413]:
seqs = parseFasta('./CS455_proj/QCBFinalProject/control/data.fa')

In [414]:
# store all substr sequences of width w for each sequence
substrings = []
substrings2 =[]
for seq in seqs:
    i = 0
    endSearch = len(seq) - width + 1
    while (i < endSearch):
        substrings.append(seq[i:i+width])
        substrings2.append(seq[i:i+width])
        i = i + 1
    substrings.append("endSeq")

In [416]:
numSeqs = len(seqs)
lenSubs = len(substrings)
numSubstrs = len(substrings2)
strScores = [[0] * numSeqs for i in range(numSubstrs)]
j = 0
substrID = 0
while (j < lenSubs):
    j, strScores, substrID, compSeqID = findHDsubstr(j, strScores, substrID, lenSubs)

In [417]:
# find total scores for each substr
finScores = []
for substr in range(0,len(strScores)):
    finScores.append(sum(strScores[substr]))

In [419]:
# find substring with the minimal Hamming distance
minHamDist = min(finScores)
indexMinHD = finScores.index(minHamDist)
motif = substrings2[indexMinHD]
motif

'ACTATAAAGA'

In [421]:
# now that we have the motif, get the PWM 
# PWM in order of A,T,C,G

# get the motif matches from each sequence
k = 0
matches = []
while (k < lenSubs):
    foundMotif, k = locMotif(k, motif)
    matches.append(foundMotif)

In [422]:
temppwm = findPWM(motif)
pwm = makeFreq(temppwm, numSeqs)
#pwm

[[0.965, 0.01, 0.015, 0.01],
 [0.005, 0.005, 0.98, 0.01],
 [0.01, 0.96, 0.02, 0.01],
 [0.99, 0.0, 0.0, 0.01],
 [0.005, 0.99, 0.0, 0.005],
 [0.955, 0.015, 0.015, 0.015],
 [0.97, 0.01, 0.01, 0.01],
 [0.975, 0.02, 0.005, 0.0],
 [0.005, 0.02, 0.005, 0.97],
 [0.96, 0.02, 0.01, 0.01]]

In [423]:
HOMEDIR = '/Users/vedranaivezic/Documents/Princeton/Princeton20-21/COS455/'
HOMEDIR = HOMEDIR + 'CS455_proj/'
writeJSON(HOMEDIR+'controlBruteSub.json', width,pwm)